In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import glob
import os
import time
import pickle
import datetime
import networkx as nx

import sys
sys.path.append('/home/ngrav/project/')
from wearables.scripts import utils as wearutils
from wearables.scripts import data as weardata
from wearables.scripts import train as weartrain
from wearables.scripts import eval_ as weareval
from wearables.scripts import model as wearmodels
from wearables.scripts import DTW as weardtw
from wearables.scripts import error_analysis as wearerr

import torch
import torch.nn as nn
import torch.nn.functional as F

from scipy.stats import spearmanr
from scipy.stats import chi2_contingency
from scipy.stats import kruskal
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sktime.datatypes._panel._convert import from_2d_array_to_nested
from sktime.regression.compose._ensemble import ComposableTimeSeriesForestRegressor
from sktime.classification.compose import ComposableTimeSeriesForestClassifier

import matplotlib.pyplot as plt
import seaborn as sns
from adjustText import adjust_text
plt.rc('font', size = 9)
plt.rc('font', family='sans serif')
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['legend.frameon']=False
plt.rcParams['axes.grid']=False
plt.rcParams['legend.markerscale']=1
plt.rcParams['savefig.dpi'] = 600
sns.set_style("ticks")

def loadpkl(fp):
    with open(fp, 'rb') as f:
        return pickle.load(f)

In [2]:
# load data

pfp = '/home/ngrav/project/wearables/results/'
mfp = '/home/ngrav/scratch/wearables_model_zoo'

# filepaths to bst or pre-processed md with calculated metrics 
pp_md_fp = os.path.join(pfp, 'md_v522_220124.csv')
bst_trainer = os.path.join(mfp, 'trainer_itv71_InceptionTime_GA5.pkl')
bst_modelpkl = os.path.join(mfp, '275-itv71_InceptionTime_GA4.pkl')

# load up to date md
md = pd.read_csv(pp_md_fp, index_col=0)
trainer = loadpkl(bst_trainer)

if False:
    # call eval to get output, modify for return_hidden = True option
    evaluation = weareval.eval_trained(trainer, split='train',
                                       modelpkl=bst_modelpkl,
                                       two_outputs=True)

In [9]:
len(trainer.data.train_ids)

1399

In [8]:
len(trainer.data.test_ids)

690

In [11]:
len(trainer.data.val_ids)

216

In [13]:
def print_n(uids):
    grp = []
    for uid in uids:
        parts = uid.split('_')
        pid = int(parts[0])
        grp.append(pid)
    print('n_grp: {}\tn_samples: {}'.format(len(np.unique(grp)), len(uids)))
    return grp

In [18]:
train_pid = print_n(trainer.data.train_ids)
test_pid = print_n(trainer.data.test_ids)
val_pid = print_n(trainer.data.val_ids)

n_grp: 658	n_samples: 1399
n_grp: 325	n_samples: 690
n_grp: 100	n_samples: 216


In [20]:
print('N_grp: {}\tN_samples: {}'.format(658+325+100, 1399+690+216))

N_grp: 1083	N_samples: 2305


In [19]:
# check why 1260 got filtered out


1083